# Cálculo de Energia Livre em Catálise usando VASP e ASE

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) <br>
Adaptado: [PhD. Hugo de Lacerda C. Neto](https://github.com/hugo-neto)

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/) <br>
Contato: [hneto@peq.coppe.ufrj.br](mailto:hneto@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/hugo/Documents/Programs/VASP/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu'
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

# Importando o VASP calculator do ASE
from ase.calculators.vasp import Vasp

from ase import Atoms
from ase.io import write, read
from ase.visualize import view

import numpy as np

# Water Gas Shift Reaction

$H_2 O + CO \to CO_2 + H_2$ 

Valores experimentais

$\Delta G_r = -28.6\ \text{kJ/mol}$

$\Delta H_r = -41.2\ \text{kJ/mol}$

In [3]:
h2o = Atoms('H2O',
            positions=[[-0.768, 0.000, 0.595],
                       [0.768, 0.000, 0.595],
                       [0.000, 0.000, 0.000]]) # geometria otimizada com PBE
h2o.center(vacuum=6.0)  
h2o.pbc = True

calc_relax = Vasp(
    directory='h2o_relax',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-6, ediffg=-0.01,
    isif=2, ibrion=2, nsw=100,
    nelm=100,
    lwave=True, lcharg=True, lvtot=True,
    atoms=h2o
)
calc_relax.calculate(h2o) # demora por volta de 10 s

E_h2o = h2o.get_potential_energy()

print(f'Optimized energy of H2O: {E_h2o:.3f} eV')

Optimized energy of H2O: -14.219 eV


In [4]:
calc_vib = Vasp(
    directory='h2o_vib',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=6,           # finite differences with symmetry
    lreal=False,
    lwave=False, lcharg=False,
    atoms=h2o
)

calc_vib.calculate(h2o)

vib_water = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_water.get_frequencies()

print(f'Vibrational frequencies of H2O (cm⁻¹):')
vib_energies_h2o = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_h2o.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of H2O (cm⁻¹):
f_v: 1584.770 cm⁻¹, E_v: 0.196 eV
f_v: 3727.986 cm⁻¹, E_v: 0.462 eV
f_v: 3840.037 cm⁻¹, E_v: 0.476 eV


In [7]:
co = Atoms('CO',
            positions=[[0.000, 0.000, 0.000],
                       [0.000, 0.000, 1.144]]) # geometria otimizada com PBE
co.center(vacuum=6.0)  
co.pbc = True

calc_relax = Vasp(
    directory='co_relax',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-6, ediffg=-0.01,
    isif=2, ibrion=2, nsw=100,
    nelm=100,
    lwave=True, lcharg=True, lvtot=True,
    atoms=co
)
calc_relax.calculate(co) # demora por volta de 10 s

E_co = co.get_potential_energy()

print(f'Optimized energy of CO: {E_co:.3f} eV')

Optimized energy of CO: -14.780 eV


In [8]:
calc_vib = Vasp(
    directory='co_vib',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=6,           # finite differences with symmetry
    lreal=False,
    lwave=False, lcharg=False,
    atoms=co
)

calc_vib.calculate(co)

vib_co = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_co.get_frequencies()

print(f'Vibrational frequencies of CO (cm⁻¹):')
vib_energies_co = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_co.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of CO (cm⁻¹):
f_v: 2125.767 cm⁻¹, E_v: 0.264 eV


In [10]:
co2 = Atoms('CO2',
            positions=[[0.000, 0.000, 0.000],
                       [0.000, 0.000, 1.177],
                       [0.000, 0.000, -1.177]]) # geometria otimizada com PBE
co2.center(vacuum=6.0)  
co2.pbc = True

calc_relax = Vasp(
    directory='co2_relax',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-6, ediffg=-0.01,
    isif=2, ibrion=2, nsw=100,
    nelm=100,
    lwave=True, lcharg=True, lvtot=True,
    atoms=co2
)
calc_relax.calculate(co2) # demora por volta de 10 s

E_co2 = co2.get_potential_energy()

print(f'Optimized energy of CO2: {E_co2:.3f} eV')

Optimized energy of CO2: -22.960 eV


In [11]:
calc_vib = Vasp(
    directory='co2_vib',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=6,           # finite differences with symmetry
    lreal=False,
    lwave=False, lcharg=False,
    atoms=co2
)

calc_vib.calculate(co2)

vib_co2 = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_co2.get_frequencies()

print(f'Vibrational frequencies of CO2 (cm⁻¹):')
vib_energies_co2 = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_co2.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of CO2 (cm⁻¹):
f_v: 631.459 cm⁻¹, E_v: 0.078 eV
f_v: 631.459 cm⁻¹, E_v: 0.078 eV
f_v: 1317.667 cm⁻¹, E_v: 0.163 eV
f_v: 2365.977 cm⁻¹, E_v: 0.293 eV


In [12]:
h2 = Atoms('H2',
            positions=[[0.000, 0.000, 0.000],
                       [0.000, 0.000, 0.740]]) # geometria otimizada com PBE
h2.center(vacuum=6.0)  
h2.pbc = True

calc_relax = Vasp(
    directory='h2_relax',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-6, ediffg=-0.01,
    isif=2, ibrion=2, nsw=100,
    nelm=100,
    lwave=True, lcharg=True, lvtot=True,
    atoms=h2
)
calc_relax.calculate(h2) # demora por volta de 10 s

E_h2 = h2.get_potential_energy()

print(f'Optimized energy of H2: {E_h2:.3f} eV')

Optimized energy of H2: -6.766 eV


In [ ]:
calc_vib = Vasp(
    directory='h2_vib',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=6,           # finite differences with symmetry
    lreal=False,
    lwave=False, lcharg=False,
    atoms=h2
)

calc_vib.calculate(h2)

vib_h2 = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_h2.get_frequencies()

print(f'Vibrational frequencies of H2 (cm⁻¹):')
vib_energies_h2 = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_h2.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of H2 (cm⁻¹):
f_v: 4320.547 cm⁻¹, E_v: 0.536 eV


Calculando energia livre de Gibbs e Entalpia

In [19]:
from ase.thermochemistry import IdealGasThermo

h2o_thermo = IdealGasThermo(vib_energies=vib_energies_h2o,
                        potentialenergy=E_h2o, atoms=h2o,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

co_thermo = IdealGasThermo(vib_energies=vib_energies_co,
                        potentialenergy=E_co, atoms=co,
                        geometry='linear', symmetrynumber=1, spin=0)

h2_thermo = IdealGasThermo(vib_energies=vib_energies_h2,
                        potentialenergy=E_h2, atoms=h2,
                        geometry='linear', symmetrynumber=2, spin=0)

co2_thermo = IdealGasThermo(vib_energies=vib_energies_co2,
                        potentialenergy=E_co2, atoms=co2,
                        geometry='linear', symmetrynumber=2, spin=0)


P = 101325. # Pa
T = 298.15 # K

G_h2o = h2o_thermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_h2o = h2o_thermo.get_enthalpy(temperature=T, verbose=False)

G_co = co_thermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_co = co_thermo.get_enthalpy(temperature=T, verbose=False)

G_h2 = h2_thermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_h2 = h2_thermo.get_enthalpy(temperature=T, verbose=False)

G_co2 = co2_thermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_co2 = co2_thermo.get_enthalpy(temperature=T, verbose=False)

# Cálculo da variação de energia livre de Gibbs e entalpia para a reação WGS
# CO + H2O -> CO2 + H2
Gwgs = G_co2 + G_h2 - (G_co + G_h2o)
Hwgs = H_co2 + H_h2 - (H_co + H_h2o)

print('Water-gas shift reaction: CO + H2O -> CO2 + H2')
print(f'at T={T} K and P={P} Pa:')
print(f'ΔG = {Gwgs:.3f} eV = {Gwgs*96.485:.3f} kJ/mol')
print(f'ΔH = {Hwgs:.3f} eV = {Hwgs*96.485:.3f} kJ/mol')


Water-gas shift reaction: CO + H2O -> CO2 + H2
at T=298.15 K and P=101325.0 Pa:
ΔG = -0.727 eV = -70.151 kJ/mol
ΔH = -0.857 eV = -82.648 kJ/mol


# Reação de WGS com Catalisador de Pt(111)

Reações de Adsorção

$ H_2 O + ^* \rightleftarrows H_2 O^*$ (1)

$ CO + * \to CO*$ (2) 

$ CO2 + * \to CO2*$ (3) 

$ H2 + * \to 2 H*$ (4) 


**Ref:** 
- Grabow, L. C., Gokhale, A. A., Evans, S. T., Dumesic, J. A., & Mavrikakis, M. (2008). _Mechanism of the water gas shift reaction on Pt: First principles, experiments, and microkinetic modeling._ The Journal of Physical Chemistry C, 112(12), 4608-4617. https://pubs.acs.org/doi/pdf/10.1021/jp7099702
- Gokhale, A. A., Dumesic, J. A., & Mavrikakis, M. (2008). _On the mechanism of low-temperature water gas shift reaction on copper._ Journal of the American Chemical Society, 130(4), 1402-1414. https://pubs.acs.org/doi/pdf/10.1021/ja0768237

## Equilibrando Geometria do Catalisador

Cristal bulk de Pt

In [21]:
from ase.lattice.cubic import FaceCenteredCubic

# bulk system
Pt_crystal = FaceCenteredCubic(size=(1, 1, 1), symbol='Pt', latticeconstant=3.967, pbc=True)

calc_relax = Vasp(directory='Pt_relax',
                xc='PBE',
                kpts=[6, 6, 6],  # specifies k-points
                encut=450,
                ismear=0, sigma=0.05,
                ediff=1e-6, ediffg=-0.01,
                isif=7, ibrion=2, nsw=100, # ionic relaxation (ISIF=7: relax cell volume only)
                atoms=Pt_crystal
                )
calc_relax.calculate(Pt_crystal) # demora por volta de 2 s

E_Pt = Pt_crystal.get_potential_energy()

a0_Pt = Pt_crystal.cell.cellpar()[0]

print(f'Optimized energy of Pt: {E_Pt:.3f} eV')
print(f"Lattice length: {a0_Pt:.3f} Å")

Optimized energy of Pt: -24.458 eV
Lattice length: 3.967 Å


Superfície Pt111

In [ ]:
from ase.build import fcc111

# Superfície Pt111
Pt111_slab = fcc111("Pt", size=(2,2,3), a=a0_Pt)
Pt111_slab.center(vacuum=5.0, axis=2)
Pt111_slab.pbc = True

calc_relax = Vasp(directory='Pt111_slab_relax',
                xc='PBE',
                kpts=[6, 6, 1],  # specifies k-points
                encut=450,
                ismear=0, sigma=0.05,
                ediff=1e-6, ediffg=-0.01,
                ibrion=-1, # just SCF (no ionic relaxation)
                atoms=Pt111_slab
                )
calc_relax.calculate(Pt111_slab) # demora por volta de 12 s

E_Pt111 = Pt111_slab.get_potential_energy()

print(f'Optimized energy of Pt111: {E_Pt111:.3f} eV')

Optimized energy of Pt111: -67.924 eV


## Reação 1: Water Adsorption

$ H_2 O + * \to H_2 O*$ 

Calculando equilíbrio da geometria da molécula $H_2 O$ adsorvida na superfície Pt111

In [ ]:
from ase.constraints import FixAtoms
from ase.build import add_adsorbate
# Adsorvendo a molécula de H2O na superfície Pt111

H2Oads_slab = fcc111("Pt", size=(2,2,3), a=a0_Pt)
H2Oads_slab.center(vacuum=5.0, axis=2)
H2Oads_slab.pbc = True

h2o = Atoms('H2O',
            positions=[[-0.768, 0.595, 0.000],
                       [0.768, 0.595, 0.000],
                       [0.000, 0.000, 0.000]]) # geometria otimizada com PBE

h2o.rotate('z', 104.46/2) # rotaciona a molécula em 60 graus em torno do eixo z

add_adsorbate(H2Oads_slab,  adsorbate=h2o, height=2.45, position = (4.50, 2.18), mol_index=2 )

# fixando os átomos de Pt
constraint = FixAtoms(mask=[atom.symbol=='Pt' for atom in H2Oads_slab])
H2Oads_slab.set_constraint([constraint])

calc_relax = Vasp(directory='Pt111_H2O_relax',
                xc='PBE',
                kpts=[6, 6, 1],  # specifies k-points
                encut=450,
                ismear=0, sigma=0.05,
                ediff=1e-6, ediffg=-0.01,
                isif=0, ibrion=2, nsw=100, # ionic relaxation (ISIF=2: relax atoms position only)
                atoms=H2Oads_slab
                )
calc_relax.calculate(H2Oads_slab) # demora por volta de 260 min

E_Pt111_H2O= H2Oads_slab.get_potential_energy()

print(f'Optimized energy of H2O in top position of Pt111: {E_Pt111_H2O:.3f} eV')


In [ ]:
E_ads_Pt111_H2O = E_Pt111_H2O - E_Pt111 - E_h2o

print(f'Adsorption energy of H2O on Pt111: {E_ads_Pt111_H2O:.3f} eV')

In [ ]:
calc_vib = Vasp(
    directory='Pt111_H2O_vib',
    xc='PBE',
    kpts=[6, 6, 1],
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=5,nfree=2, nsw=1,           # finite differences with selective dynamics (Pt atoms fixed)
    lreal=False,lwave=False, lcharg=False,
    atoms=H2Oads_slab
)

calc_vib.calculate(H2Oads_slab)

vib_Pt111_h2o = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_Pt111_h2o.get_frequencies()

print(f'Vibrational frequencies of H2O on Pt111 (cm⁻¹):')

vib_energie_Pt111_h2o = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energie_Pt111_h2o.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

In [ ]:
h2o_Pt111_thermo = IdealGasThermo(vib_energies=vib_energie_Pt111_h2o,
                        potentialenergy=E_Pt111_H2O, atoms=H2Oads_slab,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

G_h2o_Pt111 = h2o_Pt111_thermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_h2o_Pt111 = h2o_Pt111_thermo.get_enthalpy(temperature=T, verbose=False)

In [ ]:
Hads_h2o = H_h2o_Pt111 - H_h2o - E_Pt111
      
print(f'ΔH h2o adsorption = {Hads_h2o:.3f} eV')

## Reação 2: Water Dessociation

$ H_2 O(ads) \to OH(ads) + H(ads)$ 

## Resultado Final


| Reação                                 | $\Delta E$ (eV)  | $\Delta H$ (eV) |
|----------------------------------------|----------|------------|
| $ H_2 O + ^* \rightleftarrows H_2 O^*$ |  -0.210  | -0.156     |
| $ H_2 O(ads)\rightleftarrows OH(ads) + H(ads)$ |  -?.???  | -?.???     |
